# Function

In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import norm

In [ ]:
def Black_Schole(spot_price, strike_price, dividend_yield, volatility ,risk_free_rate,time_to_maturity):
  d1 = (np.log(spot_price / strike_price) + (dividend_yield + (volatility ** 2) / 2) * time_to_maturity) / (volatility * np.sqrt(time_to_maturity))
  d2 = d1 - volatility * np.sqrt(time_to_maturity)
  N_d1 = norm.cdf(d1)
  N_d2 = norm.cdf(d2)
  call_option_price = spot_price * np.exp((dividend_yield - risk_free_rate) * time_to_maturity) * norm.cdf(d1) - strike_price * np.exp(-risk_free_rate * time_to_maturity) * norm.cdf(d2)
  call_option_delta = np.exp((dividend_yield - risk_free_rate) * time_to_maturity) * norm.cdf(d1)
  put_option_price = spot_price * np.exp(-risk_free_rate * time_to_maturity) * norm.cdf(-d2) - strike_price*np.exp((dividend_yield-risk_free_rate)*time_to_maturity) * norm.cdf(-d1)
  put_option_delta = -np.exp((dividend_yield-risk_free_rate)*time_to_maturity) * norm.cdf(-d1)
  return d1, d2, N_d1, N_d2, call_option_price, call_option_delta, put_option_price, put_option_delta


In [ ]:
import numpy as np

def generate_asset_price(spot_price, d, u, num_steps):
    x = np.zeros((num_steps + 1, num_steps + 1))

    for i in range(num_steps + 1):
        for j in range(num_steps + 1):
            if i == 0 and j == 0:
                x[i, j] = spot_price
            elif i == j:
                x[i, j] = x[i-1, j-1] * d
            elif i < j:
                x[i, j] = x[i, j-1] * u
            else:
                x[i, j] = 0

    return x

In [ ]:
import numpy as np

def generate_european_call(spot_price, strike_price, d, u, q, r, num_steps):
    x = np.zeros((num_steps + 1, num_steps + 1))
    y = generate_asset_price(spot_price, d, u, num_steps)

    for i in range(num_steps + 1):
        for j in range(num_steps + 1):
            if j == num_steps:
                x[i, j] = max(0, y[i, j] - strike_price)

    n = 1
    for k in range(num_steps + 1 - n):
        for i in range(num_steps -n +1):
            for j in range(num_steps):
                if j <= num_steps - 1 and i <= num_steps - 1and i <= j:
                    x[i, j] = (q * x[i, j+1] + (1 - q) * x[i+1, j+1]) / r
        n += 1
    return x

In [ ]:
import numpy as np

def generate_american_call(spot_price, strike_price, d, u, q, r, num_steps):
    x = np.zeros((num_steps + 1, num_steps + 1))
    y = generate_asset_price(spot_price, d, u, num_steps)

    for i in range(num_steps + 1):
        for j in range(num_steps + 1):
            if j == num_steps:
                x[i, j] = max(0, y[i, j] - strike_price)

    n = 1
    for k in range(num_steps + 1 - n):
        for i in range(num_steps -n + 1):
            for j in range(num_steps):
                if j <= num_steps - 1 and i <= num_steps - 1 and i <= j:
                    x[i, j] = np.maximum(y[i,j] -  strike_price,(q * x[i, j+1] + (1 - q) * x[i+1, j+1]) / r)

        n += 1
    return x

In [ ]:
import numpy as np

def generate_european_put(spot_price, strike_price, d, u, q, r, num_steps):
    x = np.zeros((num_steps + 1, num_steps + 1))
    y = generate_asset_price(spot_price, d, u, num_steps)

    for i in range(num_steps + 1):
        for j in range(num_steps + 1):
            if j == num_steps:
                x[i, j] = max(0, strike_price - y[i, j])

    n = 1
    for k in range(num_steps + 1 - n):
        for i in range(num_steps -n +1):
            for j in range(num_steps):
                if j <= num_steps - 1 and i <= num_steps - 1and i <= j:
                    x[i, j] = (q * x[i, j+1] + (1 - q) * x[i+1, j+1]) / r
        n += 1
    return x

In [ ]:
import numpy as np


def generate_american_put(spot_price, strike_price, d, u, q, r, num_steps):
    x = np.zeros((num_steps + 1, num_steps + 1))
    y = generate_asset_price(spot_price, d, u, num_steps)

    for i in range(num_steps + 1):
        for j in range(num_steps + 1):
            if j == num_steps:
                x[i, j] = max(0, strike_price - y[i, j])

    n = 1
    for k in range(num_steps + 1 - n):
        for i in range(num_steps -n + 1):
            for j in range(num_steps):
                if j <= num_steps - 1 and i <= num_steps - 1 and i <= j:
                    x[i, j] = np.maximum(strike_price - y[i,j],(q * x[i, j+1] + (1 - q) * x[i+1, j+1]) / r)

        n += 1
    return x

In [ ]:
import numpy as np
from scipy.stats import norm

def monte_carlo_sim_option(spot_price, risk_free_rate, dividend_yield, volatility, time_to_maturity, num_paths):
  simulation = np.zeros((num_paths + 1, 1))

  for i in range(1, num_paths + 1):
    drift = (risk_free_rate - dividend_yield - 0.5 * volatility**2) * time_to_maturity
    vol_term = volatility * np.sqrt(time_to_maturity)
    random_number = np.random.rand()
    sim_price = spot_price * np.exp(drift + vol_term * norm.ppf(random_number))

    simulation[i-1, 0] = sim_price

  return simulation

# Run

In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import norm
# Example usage:
spot_price = 3500   # Current stock price
strike_price = 3500   # Option strike price
time_to_maturity = 10  # Time to option expiration in years
volatility = 0.24   # Volatility of the underlying stock
risk_free_rate = 0.084 # Risk-free interest rate
dividend_yield =  risk_free_rate - 0


In [ ]:
d1, d2, N_d1, N_d2, call_option_price, call_option_delta, put_option_price, put_option_delta = Black_Schole(spot_price, strike_price, dividend_yield, risk_free_rate, volatility, time_to_maturity)

In [ ]:
bs_model = BlackScholes(spot_price, strike_price, dividend_yield, volatility, risk_free_rate, time_to_maturity)
print("Call Option Price:", bs_model.calculate_call_option_price())
print("Put Option Price:", bs_model.calculate_put_option_price())

Call Option Price: 2101.7361567012576
Put Option Price: 112.72298870303644


In [ ]:
bs_model

In [ ]:
d1_value = bs_model.calculate_d1()
print("d1 value:", d1_value)

d1 value: 1.4862705002791383


In [ ]:
call_option_price = 400
Black_Schole(spot_price, strike_price, dividend_yield, risk_free_rate, volatility, time_to_maturity)

(3.295093321895451,
 3.029461998441307,
 0.9995080550015268,
 0.9987750513866691,
 417.9905357362433,
 0.2100326958115388,
 0.027123046510198034,
 -0.00010337538922602482)

In [ ]:
print(f"d1 is {d1:.3f}\nd2 is {d2:.3f}")
print(f"N(d1) is {N_d1:.3f}\nN(d2) is {N_d2:.3f}")
print(f"Call Value is {call_option_price:.3f}\nCall Delta is {call_option_delta:.3f}")
print(f"Put Value is {put_option_price:.3f}\nPut Delta is {put_option_delta:.3f}")

d1 is 3.295
d2 is 3.029
N(d1) is 1.000
N(d2) is 0.999
Call Value is 417.991
Call Delta is 0.210
Put Value is 0.027
Put Delta is -0.000


In [ ]:
d1, d2, N_d1, N_d2, call_option_price, call_option_delta, put_option_price, put_option_delta = Black_Schole(spot_price, strike_price, dividend_yield, risk_free_rate, volatility, time_to_maturity)

In [ ]:
import numpy as np
from scipy.stats import norm
from scipy.optimize import fsolve

def implied_parameter_calculator(parameter_value, variable, target_option_price, *args):
    # Unpack args correctly
    spot_price, strike_price, dividend_yield, time_to_maturity, risk_free_rate, volatility, option_type = args

    # Set the parameter value based on the variable being solved for
    if variable == 'spot_price':
        spot_price = parameter_value
    elif variable == 'strike_price':
        strike_price = parameter_value
    elif variable == 'dividend_yield':
        dividend_yield = parameter_value
    elif variable == 'time_to_maturity':
        time_to_maturity = parameter_value
    elif variable == 'risk_free_rate':
        risk_free_rate = parameter_value
    else:
        raise ValueError(f"Invalid variable: {variable}")

    # Black-Scholes calculations
    d1 = (np.log(spot_price / strike_price) + (dividend_yield + (volatility ** 2) / 2) * time_to_maturity) / (volatility * np.sqrt(time_to_maturity))
    d2 = d1 - volatility * np.sqrt(time_to_maturity)

    if option_type == 'call':
        option_price = spot_price * np.exp((dividend_yield - risk_free_rate) * time_to_maturity) * norm.cdf(d1) - strike_price * np.exp(-risk_free_rate * time_to_maturity) * norm.cdf(d2)
    elif option_type == 'put':
        option_price = strike_price * np.exp(-risk_free_rate * time_to_maturity) * norm.cdf(-d2) - spot_price * np.exp((dividend_yield - risk_free_rate) * time_to_maturity) * norm.cdf(-d1)
    else:
        raise ValueError("Invalid option_type. Use 'call' or 'put'.")

    return option_price - target_option_price



# Target option price
target_option_price = 400

# Choose variable to solve for
variable_to_solve = 'risk_free_rate'

# Initial guess for the parameter value
initial_guess = risk_free_rate  # or any other appropriate initial guess

# Using fsolve to find the implied parameter value
implied_parameter_result = fsolve(implied_parameter_calculator, initial_guess, args=(variable_to_solve, target_option_price, spot_price, strike_price, dividend_yield, time_to_maturity, risk_free_rate, volatility, 'call'))

print(f"Implied {variable_to_solve}:", implied_parameter_result[0])


Implied risk_free_rate: 0.2499054476327511


In [ ]:
import numpy as np
"""
spot_price = 274      # Current stock price
strike_price = 225   # Option strike price
time_to_maturity = 32/252  # Time to option expiration in years
volatility = 0.52    # Volatility of the underlying stock
risk_free_rate = 0.01 # Risk-free interest rate
dividend_yield = 0.00
"""
num_steps = 240


delta_t = time_to_maturity / num_steps
u = np.exp(volatility * np.sqrt(delta_t))
d = 1 / u
r = np.exp(risk_free_rate*delta_t)
b = np.exp(dividend_yield*delta_t)
q = (b-d)/(u-d)

In [ ]:
Asset_Price = generate_asset_price(spot_price, d, u, num_steps)
Asset_Price_df = pd.DataFrame(Asset_Price)
Asset_Price_df = Asset_Price_df.round(2)
#Asset_Price_df.replace(0, '', inplace=True)
#Asset_Price_df.head(num_steps+1)
Asset_Price_df

,0,1,2,3,4,5,6,7,8,9,...,231,232,233,234,235,236,237,238,239,240
0,3500.0,3675.73,3860.29,4054.11,4257.67,4471.45,4695.96,4931.74,5179.36,5439.41,...,2.876229e+08,3.020644e+08,3.172309e+08,3.331589e+08,3.498867e+08,3.674544e+08,3.859042e+08,4.052803e+08,4.256292e+08,4.469999e+08
1,0.0,3332.67,3500.00,3675.73,3860.29,4054.11,4257.67,4471.45,4695.96,4931.74,...,2.607783e+08,2.738719e+08,2.876229e+08,3.020644e+08,3.172309e+08,3.331589e+08,3.498867e+08,3.674544e+08,3.859042e+08,4.052803e+08
2,0.0,0.00,3173.34,3332.67,3500.00,3675.73,3860.29,4054.11,4257.67,4471.45,...,2.364392e+08,2.483107e+08,2.607783e+08,2.738719e+08,2.876229e+08,3.020644e+08,3.172309e+08,3.331589e+08,3.498867e+08,3.674544e+08
3,0.0,0.00,0.00,3021.62,3173.34,3332.67,3500.00,3675.73,3860.29,4054.11,...,2.143717e+08,2.251352e+08,2.364392e+08,2.483107e+08,2.607783e+08,2.738719e+08,2.876229e+08,3.020644e+08,3.172309e+08,3.331589e+08
4,0.0,0.00,0.00,0.00,2877.16,3021.62,3173.34,3332.67,3500.00,3675.73,...,1.943639e+08,2.041228e+08,2.143717e+08,2.251352e+08,2.364392e+08,2.483107e+08,2.607783e+08,2.738719e+08,2.876229e+08,3.020644e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.000000e-02,4.000000e-02,4.000000e-02,4.000000e-02,4.000000e-02
237,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.000000e-02,3.000000e-02,4.000000e-02,4.000000e-02
238,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.000000e-02,3.000000e-02,3.000000e-02
239,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.000000e-02,3.000000e-02


In [ ]:
euro_call = generate_european_call(spot_price, strike_price ,d, u, q, r ,num_steps)
euro_call_df = pd.DataFrame(euro_call)
euro_call_df = euro_call_df.round(2)
#euro_call_df.replace(0, '', inplace=True)
#euro_call_df.head(num_steps+1)

In [ ]:
am_call = generate_american_call(spot_price, strike_price ,d, u, q, r ,num_steps)
am_call_df = pd.DataFrame(am_call)
am_call_df = am_call_df.round(2)
#am_call_df.replace(0, '', inplace=True)
#am_call_df.head(num_steps+1)

In [ ]:
euro_put = generate_european_put(spot_price, strike_price ,d, u, q, r ,num_steps)
euro_put_df = pd.DataFrame(euro_put)
euro_put_df = euro_put_df.round(2)
#euro_put_df.replace(0, '', inplace=True)
#euro_put_df.head(num_steps+1)

In [ ]:
am_put = generate_american_put(spot_price, strike_price ,d, u, q, r ,num_steps)
am_put_df = pd.DataFrame(am_put)
am_put_df = am_put_df.round(2)
#am_put_df.replace(0, '', inplace=True)
#am_put_df.head(num_steps+1)

In [ ]:
euro_call_binomial = euro_call_df[0][0]
am_call_binomial = am_call_df[0][0]
euro_put_binomial = euro_put_df[0][0]
am_put_binomial = am_put_df[0][0]
print(f"Binomial European Call Value: {euro_call_binomial: .3f}\nBinomial American Call Value: {am_call_binomial: .3f}")
print(f"Binomial European Put Value: {euro_put_binomial: .3f}\nBinomial American Put Value: {am_put_binomial: .3f}")

Binomial European Call Value:  2100.910
Binomial American Call Value:  2100.910
Binomial European Put Value:  111.890
Binomial American Put Value:  365.150


In [ ]:
"""
spot_price = 274      # Current stock price
strike_price = 225   # Option strike price
time_to_maturity = 32/252  # Time to option expiration in years
volatility = 0.52    # Volatility of the underlying stock
risk_free_rate = 0.01 # Risk-free interest rate
dividend_yield = 0.00
num_steps = 10
"""

'\nspot_price = 274      # Current stock price\nstrike_price = 225   # Option strike price\ntime_to_maturity = 32/252  # Time to option expiration in years\nvolatility = 0.52    # Volatility of the underlying stock\nrisk_free_rate = 0.01 # Risk-free interest rate\ndividend_yield = 0.00\nnum_steps = 10\n'

In [ ]:
num_paths = 10000
simulation_price = monte_carlo_sim_option(spot_price, risk_free_rate, dividend_yield, volatility, time_to_maturity, num_paths)
Call_Expired_Value = np.maximum(0, simulation_price - strike_price)
Call_Option =  np.exp((dividend_yield -risk_free_rate)*time_to_maturity) * Call_Expired_Value.mean()
Put_Expired_Value = np.maximum(0, strike_price - simulation_price)
Put_Option =  np.exp((dividend_yield -risk_free_rate)*time_to_maturity) * Put_Expired_Value.mean()
print(f"Monte Carlo call value is {Call_Option:.2f}\nMonte Carlo put value is {Put_Option:.2f}")

Monte Carlo call value is 1018.79
Monte Carlo put value is 1041.20


In [ ]:
import numpy as np

# Monte Carlo simulation
def monte_carlo(spot_price, strike_price, dividend_yield ,risk_free_rate, volatility, time_to_maturity, num_steps):
    dt = time_to_maturity/ num_steps # Time step
    S = np.empty(num_steps+1) # Stock price
    S[0] = spot_price
    for i in range(num_steps):
        S[i+1] = S[i] * np.exp((r - dividend_yield - volatility**2 / 2) * dt + volatility * np.sqrt(dt) * np.random.normal())
    return np.maximum(S[-1] - strike_price, 0) # Payoff of call option


# Run Monte Carlo simulation
M = 1000 # Number of simulations
payoffs = np.empty(M)
for i in range(M):
    payoffs[i] = monte_carlo(spot_price, strike_price, dividend_yield ,risk_free_rate, volatility, time_to_maturity, num_steps)

# Estimate option price
C = np.exp(-r * time_to_maturity) * payoffs.mean()

print(C)

1453.098041214548
